In [11]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model, Dataset
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np


dm = DataModule(
    name=["ribo500-blast"],#, 'sarah_supermodel'],
    data_type=["dms", 'shape', 'structure'],
    force_download=False,
    batch_size=64,
    num_workers=0,
    train_split=128,
    valid_split=0,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
    use_error=False,
    structure_padding_value=0,
)

dm.setup('fit')

Loading dataset from disk
Done!                            


In [8]:
from dmsensei import create_model

params = {
    'embed_dim': 192,
    'num_heads': 6,
    'hidden_dim': 768,
    'num_encoders': 12,
    'max_len': 500,
    'use_se': True,
    
}
params['dim_per_head'] = params['embed_dim'] // params['num_heads']

model = create_model(
    'ribonanza',
    params,
    lr = 1e-3,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()
for batch in dm.train_dataloader():
    pred = model.forward(batch)['dms']
    pairs = [(p.to(device), t) for p, t in zip(pred, batch.get('dms')) if t is not None]
    print(pairs[0][0].device, pairs[0][1].device)
    optimizer.zero_grad()
    loss = torch.tensor(0.)
    for p, t in pairs:
        if t is None:
            continue
        p = p[1:-1]
        mask = t != -1000.
        if not torch.sum(mask):
            continue
        p, t = p[mask], t[mask]
        if p.device != t.device:
            print('mismatch')
        loss += criterion(p, t)
    loss.backward()
    

    break

cuda:0 cuda:0


IndexError: The shape of the mask [177] at index 0 does not match the shape of the indexed tensor [175] at index 0

In [8]:
from torch.utils.data import Sampler
import numpy as np
from random import shuffle

class BySequenceLengthSampler(Sampler):

    def __init__(self, dataset,  
                bucket_boundaries, batch_size=64,):
        ind_n_len = []
        for i, p in enumerate(dataset.length):
            ind_n_len.append( (i, p) )
        self.ind_n_len = ind_n_len
        self.bucket_boundaries = bucket_boundaries
        self.batch_size = batch_size
        self.data_source = dataset
        
    def __iter__(self):
        data_buckets = dict()
        # where p is the id number and seq_len is the length of this id number. 
        for p, seq_len in self.ind_n_len:
            pid = self.element_to_bucket_id(p,seq_len)
            if pid in data_buckets.keys():
                data_buckets[pid].append(p)
            else:
                data_buckets[pid] = [p]

        for k in data_buckets.keys():

            data_buckets[k] = np.asarray(data_buckets[k])

        iter_list = []
        for k in data_buckets.keys():
            np.random.shuffle(data_buckets[k])
            iter_list += (np.array_split(data_buckets[k]
                           , int(data_buckets[k].shape[0]/self.batch_size)))
        shuffle(iter_list) # shuffle all the batches so they arent ordered by bucket
        # size
        for i in iter_list: 
            yield i.tolist() # as it was stored in an array
    
    def __len__(self):
        return len(self.data_source)
    
    def element_to_bucket_id(self, x, seq_length):
        boundaries = list(self.bucket_boundaries)
        buckets_min = [np.iinfo(np.int32).min] + boundaries
        buckets_max = boundaries + [np.iinfo(np.int32).max]
        conditions_c = np.logical_and(
          np.less_equal(buckets_min, seq_length),
          np.less(seq_length, buckets_max))
        bucket_id = np.min(np.where(conditions_c))
        return bucket_id
    
# To call it and use it in a DataLoader (Replace the batch_size and bucket_boundaries values):



In [9]:
from dmsensei import DataModule, create_model, Dataset
from torch.utils.data import DataLoader, random_split, Subset

ds = Dataset.from_local_or_download(
    name = 'yack_valid',
)

bucket_boundaries = [50,100,125,150,175,200,250,300]
batch_sizes=32

dataloader = DataLoader(ds, batch_size=1, 
                        batch_sampler=sampler, 
                        num_workers=0, 
                        
                        drop_last=False, pin_memory=False)


Loading dataset from disk
Done!                            


In [10]:
for batch in dataloader:
    print(batch['dms'].shape)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [129]:
from scipy.optimize import linear_sum_assignment
import numpy as np
import torch

class HungarianAlgorithm:
        
    def run(self, bppm, threshold=0.5, canonical_bp_only=False, sequences=None, min_hairpin_length=3):
        
        bppm = bppm.cpu().numpy()
        
        # make sure that the input dimension is batch_size x n x n
        if len(bppm.shape) == 2:
            bppm = bppm.unsqueeze(0)
        assert len(bppm.shape) == 3, "The input bppm matrix should be batch_size x n x n"
        assert bppm.shape[1] == bppm.shape[2], "The input bppm matrix should be batch_size x n x n"
        
        # run hungarian algorithm for each batch
        bp_matrix = np.zeros(bppm.shape)
        for i in range(bppm.shape[0]):
            # assert self.is_symmetric(bppm[i]), "The input bppm matrix should be symmetric"
            
            # remove non-canonical base pairs and too short hairpins
            if canonical_bp_only:
                assert sequences is not None, "If canonical_bp_only is True, sequences should be provided"
                bppm[i] = self._remove_non_canonical(bppm[i], sequences[i])
            bppm[i] = self._remove_hairpins(bppm[i], min_hairpin_length)
            
            # run hungarian algorithm only on rows and columns that have at least one value greater than threshold
            compressed_matrix_idx = self._pairable_bases(bppm[i], threshold)
            compressed_bppm = bppm[i][compressed_matrix_idx][:, compressed_matrix_idx]
            row_ind, col_ind = self._hungarian_algorithm(compressed_bppm)         
            
            # convert the result to the original sized matrix
            for compressed_row, compressed_col in zip(row_ind, col_ind):
                bppm_row, bppm_col = compressed_matrix_idx[compressed_row], compressed_matrix_idx[compressed_col]
                if bppm[i, bppm_row, bppm_col] > threshold:
                    bp_matrix[i, bppm_row, bppm_col] = 1
            
        return bp_matrix
    
    def _hungarian_algorithm(self, cost_matrix):
        row_ind, col_ind = linear_sum_assignment(-cost_matrix)
        return row_ind, col_ind
    
    def _pairable_bases(self, bppm, threshold):
        n = bppm.shape[1]
        row_ind = []
        for i in range(n):
            if bppm[i].max() > threshold: 
                row_ind.append(i)
        return row_ind
    
    def is_symmetric(self, bppm):
        return (bppm == bppm.transpose(1, 0)).all()
    
    def _remove_non_canonical(self, bp_matrix, sequence):
        """This function removes non-canonical base pairs (keeps only AU, CG, GU, and all base pairs with N)
        
        Logic: 
            - embed the sequence
            - sum the sequence with its transpose
            - canonical base pairs sum into 3, 5 or negative values (N) by design
            
            Mapping: A=0, C=1, G=2, U=3, N=-4
            Which gives, in the sum:    
                A | C | G | U | N
            A | 0 | 1 | 2 | 3 | -4
            C | 1 | 2 | 3 | 4 | -3
            G | 2 | 3 | 4 | 5 | -2
            U | 3 | 4 | 5 | 6 | -1
            N |-4 |-3 |-2 |-1 | -8
            
            If you keep only the values that are 3, 5 or -4, you keep only the canonical base pairs     
        """
        # embed the sequence
        mapping = {'A': 0, 'C': 1, 'G': 2, 'U': 3, 'T': 3, 'N': -4}
        sequence = np.array([mapping[base] for base in sequence]).reshape(-1, 1)
        
        # make the pairing matrix
        mat = sequence + sequence.T
        
        # remove non-canonical base pairs
        bp_matrix[(mat>=0) & (mat!=3) & (mat!=5)] = 0
        
        return bp_matrix
    
    def _remove_hairpins(self, bp_matrix, min_hairpin_length):
        """This function removes hairpins that are too short and self-pairing
        
        Example:
        >>> HungarianAlgorithm()._remove_hairpins(np.ones((5,5)), 2) === [[0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0]]
        True
        """
        n = bp_matrix.shape[1]
        for i in range(n):
            for j in range(i-min_hairpin_length, i+min_hairpin_length+1):
                if j >= 0 and j < n:
                    bp_matrix[i, j] = 0
        return bp_matrix


ex = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

ex += np.random.normal(0, 0.2, ex.shape)

# make symmetric
ex = (ex + ex.T) / 2
# print(ex.round(3).tolist())

HungarianAlgorithm().run(torch.tensor(ex).unsqueeze(0), threshold=0.5, canonical_bp_only=True, sequences=['GAACUAUUCU']).squeeze(0).tolist()


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [127]:

(HungarianAlgorithm()._remove_non_canonical(np.ones((5,5)), 'ACGUN') == np.array([[0., 0., 0., 1.,1.],[0.,0.,1.,0.,1.],[0.,1.,0.,1.,1.],[1.,0.,1.,0.,1.],[1.,1.,1.,1.,1.]])).all()

False

In [130]:
HungarianAlgorithm()._remove_non_canonical(np.ones((5,5)), 'ACGUN')

array([[0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 1.],
       [1., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1.]])

In [120]:
np.diag(np.ones(10))[::-1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [90]:
''.join(np.random.choice(list('ACGU'), 10))

'CCCCUAUUCU'